<a href="https://colab.research.google.com/github/lucarenz1997/recommender_systems/blob/main/Hybrid-NCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid Neural Collaborative Filtering (NCF) Model
** Authors **: Rafaella and Luca
[Linktext](https://)

## Setup

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive


## Loading and Data Preparation

In [3]:
#Load data
data = pd.read_csv("/content/drive/MyDrive/Recommender/preprocessed.csv")

#Encoding
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
gender_encoder = LabelEncoder()
platform_encoder = LabelEncoder()

data["user_id"] = user_encoder.fit_transform(data["user_id"])
data["media_id"] = item_encoder.fit_transform(data["media_id"])
data['user_gender_enc'] = gender_encoder.fit_transform(data['user_gender'])
data['platform_name_enc'] = platform_encoder.fit_transform(data['platform_name'])

# MinMax scaling for 'user_age' to bring it to a 0-1 range
scaler = MinMaxScaler()
data['user_age_scaled'] = scaler.fit_transform(data[['user_age']])
data.head(100)

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,...,media_popularity,artist_popularity,album_popularity,songs_listened,song_popularity_7d,artist_popularity_7d,album_popularity_7d,user_gender_enc,platform_name_enc,user_age_scaled
0,1175,2016-11-28 16:40:24,23564,1437691,0,2003-05-19,0,0,306,0,...,5,580,40,28,1,79,5,0,0,0.083333
1,0,2016-11-17 23:19:07,61787,13632884,0,2016-07-22,1,0,239,0,...,152,536,152,52,22,63,22,0,1,0.250000
2,2692,2016-11-23 14:20:52,19290,919739,2,2008-09-16,1,0,228,0,...,1,238,5,56,0,40,0,1,1,0.083333
3,14,2016-11-24 14:19:09,42166,8980343,4,2014-10-27,0,0,222,1,...,3,318,29,10,0,51,3,0,0,0.333333
4,0,2016-11-02 15:07:25,62038,13680778,0,2016-07-29,0,0,183,0,...,73,295,73,53,0,32,0,0,0,0.416667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,25,2016-11-12 17:54:14,12618,338342,1,2009-07-07,2,1,306,1,...,64,84,72,16,11,18,14,1,2,0.833333
96,0,2016-11-20 01:10:11,58568,13082992,2,2016-05-06,0,0,253,0,...,8,2347,1074,55,0,353,121,1,0,0.166667
97,0,2016-11-17 12:36:05,65109,14103674,0,2016-09-24,0,0,248,0,...,62,62,62,40,9,9,9,0,0,0.500000
98,7,2016-11-20 07:49:36,12302,324222,13,1995-12-31,0,0,247,1,...,19,238,21,75,1,40,3,0,0,0.750000


from matplotlib import pyplot as plt
_df_0['genre_id'].plot(kind='hist', bins=20, title='genre_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['media_id'].plot(kind='hist', bins=20, title='media_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['album_id'].plot(kind='hist', bins=20, title='album_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['context_type'].plot(kind='hist', bins=20, title='context_type')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['time_of_day'].value_counts()
    for x_label, grp in _df_22.groupby('is_weekend')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('is_weekend')
_ = plt.ylabel('time_of_day')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_23['user_gender'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_23, x='genre_id', y='user_gender', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_24['is_listened'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_24, x='genre_id', y='is_listened', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_25['is_weekend'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_25, x='genre_id', y='is_weekend', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_26['time_of_day'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_26, x='genre_id', y='time_of_day', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

##Train/Test Split

In [4]:
train, test = train_test_split(data, test_size=0.2, random_state=42)
train.head(100)

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,...,media_popularity,artist_popularity,album_popularity,songs_listened,song_popularity_7d,artist_popularity_7d,album_popularity_7d,user_gender_enc,platform_name_enc,user_age_scaled
8558,0,2016-11-17 23:56:27,48426,10770214,0,2015-07-17,0,0,255,0,...,2,30,2,53,0,6,0,0,0,0.583333
234700,7,2016-11-24 15:06:50,3606,102427,1,2007-11-26,2,1,197,1,...,64,113,102,65,13,22,20,1,2,0.666667
65135,0,2016-11-06 16:08:14,58648,13091982,0,2016-05-13,0,0,230,0,...,11,11,11,259,1,1,1,1,0,0.083333
237176,0,2016-11-18 13:07:07,60660,13457671,0,2016-07-08,1,0,197,0,...,536,1114,973,26,74,154,132,0,1,0.083333
312190,0,2016-11-23 06:37:13,64837,14079078,0,2016-09-20,0,0,194,0,...,930,7569,930,68,97,992,97,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261620,0,2016-11-12 21:03:47,59373,13246617,4,2016-07-04,0,0,251,1,...,150,7569,2125,32,20,992,275,1,0,0.750000
327784,0,2016-11-22 23:34:19,52341,11674708,0,2015-11-13,0,0,233,0,...,148,653,500,71,23,96,69,0,0,0.916667
119518,0,2016-11-12 09:40:16,68991,14530576,2,2016-11-11,1,0,194,0,...,181,5083,3111,34,27,843,642,0,1,0.166667
23657,7054,2016-11-14 15:27:59,36786,7476147,1,2014-03-03,0,0,187,1,...,2,5,3,70,0,1,1,0,0,0.583333


from matplotlib import pyplot as plt
_df_27['genre_id'].plot(kind='hist', bins=20, title='genre_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_28['media_id'].plot(kind='hist', bins=20, title='media_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_29['album_id'].plot(kind='hist', bins=20, title='album_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_30['context_type'].plot(kind='hist', bins=20, title='context_type')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['time_of_day'].value_counts()
    for x_label, grp in _df_49.groupby('is_weekend')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('is_weekend')
_ = plt.ylabel('time_of_day')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_50['user_gender'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_50, x='genre_id', y='user_gender', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_51['is_listened'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_51, x='genre_id', y='is_listened', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_52['is_weekend'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_52, x='genre_id', y='is_weekend', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_53['time_of_day'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_53, x='genre_id', y='time_of_day', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

## Create PyTorch dataset

In [5]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

class ListenDataset(Dataset):
    def __init__(self, data):
        self.users = torch.tensor(data['user_id'].values, dtype=torch.long)
        self.items = torch.tensor(data['media_id'].values, dtype=torch.long)
        self.ages = torch.tensor(data['user_age_scaled'].values, dtype=torch.float32)
        self.genders = torch.tensor(data['user_gender_enc'].values, dtype=torch.long)
        self.platforms = torch.tensor(data['platform_name_enc'].values, dtype=torch.long)
        self.labels = torch.tensor(data['is_listened'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (self.users[idx], self.items[idx], self.ages[idx], self.genders[idx], self.platforms[idx]), self.labels[idx]

# Batch-Processing
train_data = ListenDataset(train)
test_data = ListenDataset(test)

train_loader = DataLoader(train_data, batch_size=512, shuffle=True)
test_loader = DataLoader(test_data, batch_size=512, shuffle=False)


## Model Definition

In [6]:
class NCF(nn.Module):
    def __init__(self, num_users, num_items, num_genders, num_platforms, emb_size=64):
        super(NCF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.gender_emb = nn.Embedding(num_genders, emb_size // 2)  # Smaller embedding size for gender
        self.platform_emb = nn.Embedding(num_platforms, emb_size // 2)

        self.fc_layers = nn.Sequential(
            nn.Linear(emb_size * 2 + emb_size // 2 * 2 + 1, 128),  # +1 for age
            nn.ReLU(),
            nn.Dropout(0.5),  # Adding dropout for regularization
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, user, item, age, gender, platform):
        user_embedded = self.user_emb(user)
        item_embedded = self.item_emb(item)
        gender_embedded = self.gender_emb(gender)
        platform_embedded = self.platform_emb(platform)

        x = torch.cat([user_embedded, item_embedded, gender_embedded, platform_embedded, age.unsqueeze(1)], dim=-1)
        return self.fc_layers(x)


## Initiate Model

In [7]:
import torch.optim as optim

# Count unique values for all entities
num_users = data['user_id'].nunique()
num_items = data['media_id'].nunique()
num_genders = data['user_gender_enc'].nunique()
num_platforms = data['platform_name_enc'].nunique()

# Create the model instance with the new parameters
model = NCF(num_users, num_items, num_genders, num_platforms)

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss, suitable for binary labels

## Train Model

In [8]:
# Move the model to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):
    model.train()
    total_loss = 0
    for batch in train_loader:
        (user, item, age, gender, platform), label = batch
        user, item, age, gender, platform, label = user.to(device), item.to(device), age.to(device), gender.to(device), platform.to(device), label.to(device)
        optimizer.zero_grad()
        preds = model(user, item, age, gender, platform).squeeze()
        loss = criterion(preds, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.6004
Epoch 2, Loss: 0.5603
Epoch 3, Loss: 0.5234
Epoch 4, Loss: 0.4990
Epoch 5, Loss: 0.4806
Epoch 6, Loss: 0.4646
Epoch 7, Loss: 0.4499
Epoch 8, Loss: 0.4356
Epoch 9, Loss: 0.4219
Epoch 10, Loss: 0.4089


## Evaluate Model

In [20]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        # Unpack the batch into features and labels
        (user, item, age, gender, platform), label = batch

        # Move each tensor to the device
        user = user.to(device)
        item = item.to(device)
        age = age.to(device)
        gender = gender.to(device)
        platform = platform.to(device)
        label = label.to(device)

        # Forward pass through the model with all features
        preds = model(user, item, age, gender, platform).squeeze()

        # Convert predictions and labels to numpy arrays for metric calculation
        predicted = (preds >= 0.4).float()  # Binary classification threshold at 0.5
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(label.cpu().numpy())

# Calculate evaluation metrics
f1 = f1_score(all_labels, all_preds)
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)

print(f"F1-Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


F1-Score: 0.8173
Accuracy: 0.7260
Precision: 0.7516
Recall: 0.8958


In [21]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# Loop over threshold values from 0.1 to 0.9
for threshold in np.arange(0.1, 1.0, 0.1):
    all_preds, all_labels = [], []

    # Evaluate the model with the current threshold
    model.eval()
    with torch.no_grad():
        for batch in test_loader:
            # Unpack the batch into features and labels
            (user, item, age, gender, platform), label = batch

            # Move each tensor to the device
            user = user.to(device)
            item = item.to(device)
            age = age.to(device)
            gender = gender.to(device)
            platform = platform.to(device)
            label = label.to(device)

            # Forward pass through the model with all features
            preds = model(user, item, age, gender, platform).squeeze()

            # Apply the threshold to get binary predictions
            predicted = (preds >= threshold).float()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(label.cpu().numpy())

    # Calculate evaluation metrics
    f1 = f1_score(all_labels, all_preds)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)

    # Print the metrics for the current threshold
    print(f"Threshold: {threshold:.1f}")
    print(f"  F1-Score : {f1:.4f}")
    print(f"  Accuracy : {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall   : {recall:.4f}")
    print("-" * 30)


Threshold: 0.1
  F1-Score : 0.8158
  Accuracy : 0.6923
  Precision: 0.6910
  Recall   : 0.9956
------------------------------
Threshold: 0.2
  F1-Score : 0.8215
  Accuracy : 0.7112
  Precision: 0.7117
  Recall   : 0.9715
------------------------------
Threshold: 0.3
  F1-Score : 0.8213
  Accuracy : 0.7230
  Precision: 0.7353
  Recall   : 0.9301
------------------------------
Threshold: 0.4
  F1-Score : 0.8173
  Accuracy : 0.7260
  Precision: 0.7516
  Recall   : 0.8958
------------------------------
Threshold: 0.5
  F1-Score : 0.8084
  Accuracy : 0.7229
  Precision: 0.7672
  Recall   : 0.8542
------------------------------
Threshold: 0.6
  F1-Score : 0.7923
  Accuracy : 0.7146
  Precision: 0.7890
  Recall   : 0.7957
------------------------------
Threshold: 0.7
  F1-Score : 0.7455
  Accuracy : 0.6802
  Precision: 0.8185
  Recall   : 0.6844
------------------------------
Threshold: 0.8
  F1-Score : 0.6436
  Accuracy : 0.6095
  Precision: 0.8569
  Recall   : 0.5153
-----------------------